## 海洋戦略の最適化

条件
- 36yen/kwh
- 風力発電0.5yearたったら点検まで動けない
- 発電機は0.2/基/yで発生
- 修理は6-18時の状態1~3でのみできる
- 1基の点検には36時間かかる
- 修理には120時間かかる
- 作業船は現地から3時間かかる。
- 運行するかには天気予報は使えない（現在の天気のみ）
- 作業船は一隻2×10^8円かかり償却費2×10^7円　運転費は10^6円/24hかかる
- 事業期間は20year
- 発電機は200個
- 天気はmarkov_に従う(6hごとの遷移）
- 運転費は港ではかからない

最適化するもの  
`S = Σw*δ　- 2×10^8t - 20×2×10^7t - (10^6/24)×Σc`
```
w = [0,1900,5000,5000,0]
t .. 船の個数
c .. 運転時間
δ　.. 動けるかどうか
```

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.linalg

In [211]:
# milは(state,time)の二元配列
# state [0:正常,1:要点検,2:故障中,3:要点検かつ故障,５：正常かつ点検中]
# 各milは[state,点検までの時間,直して欲しい時間]
class Windmil():
    earnings = np.array([0,1900,5000,5000,0])
    def __init__(self):
        self.mil = np.zeros(600).reshape(200,3)
        self.mil[:,1] = 182*8
        self.mil[:,2] = 12
    def proceed(self,wind_type):
        gain = np.sum(self.mil[:,0] == 0)*Windmil.earnings[wind_type]
#         故障と点検までの時間を評価
        t = np.random.poisson(0.2/365/8,200)
#         故障を処理
        for i in range(self.mil.shape[0]):
            if t[i]:
                if self.mil[i][0] == 0:
                    self.mil[i] = np.array([2,self.mil[i][1],40])
                elif  self.mil[i][0] == 1:
                    self.mil[i] == np.array([3,0,self.mil[i][2]+40])
                elif  self.mil[i][0] == 5:
                    self.mil[i] == np.array([2,self.mil[i][1],self.mil[i][2]+40])
#   点検までの時間を減らす
        self.mil[self.mil[:,0] %2 == 0][:,1] -= 1
#     要点検に変える
        self.mil[np.logical_and(self.mil[:,0] == 0,self.mil[:,1] == 0)] = np.array([1,0,12])
        self.mil[np.logical_and(self.mil[:,0] == 2,self.mil[:,1] == 0)] += np.array([1,0,12])
#   稼いだ電気料金を返す
        return gain
# 修理のためのメソッド
    def repair(self,ship):
        if ship.exist:
            self.mil[ship.position][2] -=1
            if self.mil[ship.position][2] == 0:
                if self.mil[ship.position][0] == 2:
                    self.mil[ship.position] = np.array([0,self.mil[ship.position][1],12])
                    Ship.windmil_index.remove(ship.position)
#     inspectの場合はinspectedに追加
                else:
                    self.mil[ship.position] = np.array([0,182*8,12])
                    Ship.inspected.append(ship.position)
                    Ship.windmil_index.remove(ship.position)
                ship.position = None
        else :
            ship.exist = True
# 定期点検よう
    def get_inspect(self,position,ship):
        if self.mil[position][0] == 0:
            self.mil[position][0] = 5
            ship.position = position
            Ship.windmil_index.append(position)

In [200]:
class Ship():
# 今船が見ている風車
    windmil_index = []
# 定期点検したもの
    inspected = []
    def __init__(self,index):
        self.position = None
        self.exist = False
        self.idx = index
    def find(self,position):
        if self.position == None:
            self.position = position
            Ship.windmil_index.append(position)
            return True
        return False

In [56]:
markov = pd.read_csv("wind_markov.csv")

In [57]:
markov = markov.values[:,1:]

In [213]:
def simulate(n_ships = 10,n_iter = 1,span = 120 *8,inspect_number = 200):
    gains = np.zeros(n_iter)
    costs = np.zeros(n_iter)
    for t in range(n_iter):
        wind = 2
        ships = []
        for i in range(n_ships):
            ship = Ship(i)
            ships.append(ship)
        windmil = Windmil()
        gain = 0
        Ship.windmil_index = []
# 初期コストの設定
        cost = n_ships * (2*10**8 + 20*2*10**7)
        teiki = False
# 20年分をloop
        for count in range (8*365*20):
            w = markov[wind]
            p = np.random.rand(1)[0]
            for j in range(5):
                p -= w[j]
                if p <=0:
                    wind = j
                    break
#     風車の状態を変える
            gain += windmil.proceed(wind)
            if count %8 >= 2 | count%8 <= 5:
#     風を判断して船を戻す
                if wind >= 4:
                    for ship in ships:
                        if ship.exist:
                            ship.exist = False
                            cost += 10**6/8
#     windmilをrepair
                else:
                    for ship in ships:
                        if ship.position != None:
                            windmil.repair(ship)
# 故障したwindmilにshipをmatchさせる
                    should_repair = np.where(windmil.mil[:,0] == 2)[0][np.logical_not(np.isin(np.where(windmil.mil[:,0] == 2)[0] ,np.asarray(Ship.windmil_index)))]
                    repair_index = 0
                    if  should_repair.shape[0]:
                        for ship in ships:
                            if repair_index == should_repair.shape[0]:
                                break
                            if ship.find(should_repair[repair_index]):
                                repair_index += 1
# 定期点検を行う
                    if count % (span) == 0:
                        teiki = True
                        Ship.inspected = []
                    if teiki:
# まず要点検があったらそっちから
# state5もあるけどそれはあとで省かれる
                        state_1 = np.where(windmil.mil[:,0] % 2 == 1)
                        state1_index = 0
                        for ship in ships:
                            if not state_1[state1_index] in Ship.windmil_index:
                                if ship.find(state_1[state1_index]):
                                    state1_index += 1
# state1につける
                        should_inspected = np.where(windmil.mil[:,0]==0)[0]
#   残り時間が少ない順にソート
                        inspect_list = np.argsort(windmil.mil[should_inspected][2])
                        inspect_index = 0
                        for ship in ships:
                            if ship.position == None:
                                windmil.get_inspect(should_inspected[inspect_list[inspect_index]],ship)
                                inspect_index += 1
                                if  inspect_index == should_inspected.shape[0]:
                                    break
                        if len(Ship.inspected) == inspect_number:
                            teiki = False
# 運転コストを計算
                    for ship in ships:
                        if ship.exist:
                            cost += 10**6/8
# 直すwindmilの無いshipを港に戻す
                    for ship in ships:
                        if ship.position == None:
                            ship.exist = False
# 夜は船を戻す
            else:
                for ship in ships:
                    if ship.exist:
                        ship.exist = False
                        cost += 10**6/8
        gains[t] = gain
        costs[t] = cost
    return gains.mean(), costs.mean()

In [214]:
gain,cost = simulate(n_iter = 2)
print("gain:%d\ncost:%d\ndiff:%d"%(gain,cost,gain-cost))

gain:28535365850
cost:17563125000
diff:10972240850
